[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Scientific Programming Methods

## Dynamic Programming - Content Aware Image Resize by Seam Carving

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 03/05/2025 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0092DeepLearningResNet.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning

# Deep Learning

# Image Processing & Computer Vision
import skimage as ski

# Miscellaneous
from enum import auto, Enum, unique
import math
import os
from platform import python_version
import random

# Typing
from typing import Callable, Dict, Generator, List, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython


## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
vallToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Color Palettes
lMatPltLibclr   = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'] #<! Matplotlib default color palette
lFlexokiClr     = ['#D14D41', '#DA702C', '#D0A215', '#879A39', '#3AA99F', '#4385BE', '#8B7EC8', '#CE5D97'] #<! Flexoki (Obsidian) Main
lFlexokiSatClr  = ['#AF3029', '#BC5215', '#BC5215', '#66800B', '#24837B', '#205EA6', '#5E409D', '#A02F6F'] #<! Flexoki (Obsidian) Saturated
lFlexokiGrayClr = ['#100F0F', '#1C1B1B', '#282726', '#343331', '#403E3C', '#55524E', '#878580', '#CECDC3'] #<! Flexoki (Obsidian) Grayscale
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())


In [ ]:
# Constants



In [ ]:
# Courses Packages



In [ ]:
# General Auxiliary Functions

def CalcImageEnergy( mI: np.ndarray ) -> np.ndarray:
    """
    Calculate the energy of an image using the Squared L2 Norm of the gradient.
    The function sums over the channels channel.
    Input:
        - mI: The input image (Gray / RGB).
    Output:
        - mO: The energy of the image.
    Example:
        ```python
        mI = np.random.rand(100, 100, 3)  # Example RGB image
        mO = CalcImageEnergy(mI)
        ```
    """

    numRows, numCols = mI.shape[:2]
    mX = np.zeros((numRows, numCols), dtype = mI.dtype)
    mY = np.zeros((numRows, numCols), dtype = mI.dtype)

    if np.ndim(mI) == 3:
        for ii in range(mI.shape[2]):
            tuGradImg = np.gradient(mI[:, :, ii])
            mX += np.square(tuGradImg[0])
            mY += np.square(tuGradImg[1])
    else:
        tuGradImg = np.gradient(mI)
        mX += np.square(tuGradImg[0])
        mY += np.square(tuGradImg[1])

    mO = mX + mY
    
    return mO


## Content Aware Image Resizing

Given an Image:

<!-- ![](https://i.imgur.com/H6dGYT2.png) --> <!-- Original Image -->
<!-- ![](https://i.postimg.cc/0QbnWDFX/image.png) --> <!-- Original Image -->

![](https://i.imgur.com/wpvoiJf.jpeg)
Image by [Kiril Dobrev]() on [Pixabay](https://pixabay.com/photos/blue-beach-surf-travel-surfer-4145659).

Of size `(360, 640)` one would like to resize it into `(360, 540)` without creating distortions.  
Namely, though the _Aspect Ratio_ is changed, the image would look natural.


* <font color='brown'>(**#**)</font> The concept was proposed in [Shai Avidan, Ariel Shamir - Seam Carving for Content Aware Image Resizing (2007)](https://dl.acm.org/doi/10.1145/1275808.1276390).

In [ ]:
# Parameters

# Data
imgUrl = r'https://i.imgur.com/wpvoiJf.jpeg' #<! (640, 360)
imgUrl = r'https://i.postimg.cc/PqtTwr95/blue-4145659-640.jpg' #<! (640, 360)

# Model
tuOutSize = (360, 360)
λ         = 1.0


In [ ]:
# Load / Generate Data

mI = ski.io.imread(imgUrl)
mI = ski.util.img_as_float64(mI)
print(f'Image Dimensions: {mI.shape}')

In [ ]:
# Plot Image

hF, hA = plt.subplots(figsize = (10, 6))
hA.imshow(mI)
hA.set_title('Input Image');

### Naive Resizing

This section applies a naive resizing to the image.

In [ ]:
# Resize Image

mO = ski.transform.resize(mI, tuOutSize)
print(f'Output Dimensions: {mO.shape}')


In [ ]:
# Plot Image

hF, hA = plt.subplots(figsize = (10, 6))
hA.imshow(mO)
hA.set_title('Resized Image');

### Seam Carving

Instead of resampling the image over the grid, the Seam Carving approach removes unimportant data from the image.  
Instead of _cropping_ the removed data is without removing significant content from the image.  
The technique identifies _low energy_ areas of the image. The energy is measured by the Gradient Field of the image.    
The assumption is the energy of the gradient is highly correlated with the information of the area.  
Once low energy areas are found, the lowest energy _seams_ that weave through the image.

**ADD IMAGE**

### Energy Image

The Seam Carving method defines the local energy of a pixel by:

$$ e \left( x, y \right) = {\left| {\Delta}_{x} R \left( x, y \right) \right|}^{2} + {\left| {\Delta}_{y} R \left( x, y \right) \right|}^{2} + {\left| {\Delta}_{x} G \left( x, y \right) \right|}^{2} + {\left| {\Delta}_{y} G \left( x, y \right) \right|}^{2} + {\left| {\Delta}_{x} B \left( x, y \right) \right|}^{2} + {\left| {\Delta}_{y} B \left( x, y \right) \right|}^{2}  $$

* <font color='brown'>(**#**)</font> There are many variations to the local energy definition.

In [ ]:
# Calculate the Energy of the Image

mE = CalcImageEnergy(mO)
print(f'Energy Dimensions: {mE.shape}')

In [ ]:
# Plot Energy Image

hF, hA = plt.subplots(figsize = (10, 6))
hA.imshow(mE)
hA.set_title('Energy Image');

## Seam Carving for Content Aware Image Resizing

The concept of Seam Carving is all about removing content with minimal loss of content.  
Hence it focuses on removing seam along low low energy.

### Seam

* A _seam_ is sequence of pixels, exactly one per row.
* A _seam_ is contiguous. Namely it is [8 Neighborhood connected](https://en.wikipedia.org/wiki/Pixel_connectivity).
* The energy of a _seam_ is the sum of Energy along its pixels.

* <font color='brown'>(**#**)</font> The definition is in the context of a vertical _seam_. It can be defined in the context of an horizontal seam as well.

In [ ]:
# Generating Random Seam

mS1 = np.c_[np.arange(mE.shape[0]), 50 + np.cumsum(np.random.randint(low = -1, high = 2, size = mE.shape[0]))] #<! Low value
mS2 = np.c_[np.arange(mE.shape[0]), 275 + np.cumsum(np.random.randint(low = -1, high = 2, size = mE.shape[0]))] #<! High value

seamEnergy1 = np.sum(mE[mS1[:, 0].astype(int), mS1[:, 1].astype(int)])
seamEnergy2 = np.sum(mE[mS2[:, 0].astype(int), mS2[:, 1].astype(int)])

hF, hA = plt.subplots(figsize = (10, 6))
hA.imshow(mE)
hA.plot(mS1[:, 1], mS1[:, 0], color = 'cyan', linewidth = 1.5, alpha = 0.45, label = f'Seam 1: {seamEnergy1:.2f}')
hA.plot(mS2[:, 1], mS2[:, 0], color = 'magenta', linewidth = 1.5, alpha = 0.45, label = f'Seam 2: {seamEnergy2:.2f}')
hA.legend()
hA.set_title('Energy Image and Seam Carves');

* <font color='red'>(**?**)</font> Which seam contains more content?
* <font color='red'>(**?**)</font> Which seam can be removed (Set of pixels defined by the seam) with less affect on the image's content?

### Problem Statement

Given a 2D array, find a vertical seam which minimizes the sum of the array values it contains:

$$ \arg \min_{\mathcal{S}} \sum_{k = 1}^{N} E \left[ {i}_{k}, {j}_{k} \right], \; \mathcal{S} = \left\{ \left( k, j \right)_{k} \right\}_{k = 1}^{N}, \; \text{subject to} \; \left| {j}_{k + 1} - {j}_{k} \right| \leq 1 $$

Namely the set of indices $\mathcal{S}$ minimizes the sum over the array when going dows the array limited to the steps: _bottom left_, _bottom_ and _bottom right_.

## Greedy Solution

The greedy solution chooses the pixel in the next row with the lowest value.  
Yet this greedy policy, while simple, does not lead to an optimal solution.

![](https://i.imgur.com/VzcMav7.png)
<!-- ![](https://i.postimg.cc/Y0Y0BCm1/Diagrams-Seam-Carving-001.png) -->

### Dynamic Programming Solution

The _Dynamic Programming_ recursive solution is given by:

$$ {C}_{i, j} = {E}_{i, j} + \min \left\{ {C}_{i - 1, j - 1}, {C}_{i - 1, j}, {C}_{i - 1, j + 1} \right\} $$

Where 

 * The term ${C}_{i, j}$ is the sum of the cost of the optimal seam going through pixel $\left( i, j \right)$.
 * Initialization of the first row: $ {C}_{1, j} = {E}_{i, j}$.
 * The objective $min_{k \in \left\{ 1, 2, 3, \ldots, N \right\}} {C}_{M, k}$ which the optimal Seam end point.

In order to be able to the recover the seam path, it is easier to formulate the recurrence relation as:

$$ {C}_{i, j} = {E}_{i, j} + \min_{k \in \left\{ -1, 0, 1 \right\} } {C}_{i - 1, j + k} $$

![](https://i.imgur.com/jUhu4fX.png)
<!-- ![](https://i.postimg.cc/W1M4nRzd/Diagrams-Seam-Carving-002.png) -->

The matrix of the parameter `K` is the one to infer the seam path from.

![](https://i.imgur.com/m97po38.png)
<!-- ![](https://i.postimg.cc/h4XjP5nJ/image.png) -->


In [ ]:
def CalcCostMatrix( mE: np.ndarray ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculate the cost matrix for a given Energy Matrix.  
    The cost at position (i, j) is defined as C_ij = E_ij + min_{k = -1, 0, 1}C_i-1k. 
    Input:
        - mE: The input Energy Matrix (numRows, numCols).
    Output:
        - mC: The cost matrix (numRows, numCols).
        - mK: The backtrack matrix (numRows, numCols).
    Example:
        ```python
        mX = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])
        mC, mK = CalcCostMatrix(mX)
        print(mC)
        ```
    Remarks:
        - The cost matrix is used to find the optimal seam in the image.
    """
    
    numRows = mE.shape[0]
    numCols = mE.shape[1]
    mC      = mE.copy()
    mK      = np.zeros((numRows, numCols), dtype = int) #<! Backtrack matrix
    
    for ii in range(1, numRows):
        for jj in range(numCols):
            minVal = mC[ii - 1, jj]
            minKK  = 0
            for kk in range(-1, 2):
                if (jj + kk) < 0 or (jj + kk) >= numCols:
                    continue
                if mC[ii - 1, jj + kk] < minVal:
                    minVal = mC[ii - 1, jj + kk]
                    minKK  = kk
            mC[ii, jj] += minVal
            mK[ii, jj] = minKK
    
    return mC, mK

In [ ]:
# Extract the Path from `mP` Matrix

def ExtractSeamPath( mK: np.ndarray, mC: np.ndarray ) -> np.ndarray:
    """
    Extract the path from the path matrix `mP` using the minimum cost segmentation matrix `mS`.
    The function starts from the last segment and traces back to the first segment using the path matrix.
    Input:
        - mK: Path matrix (2D numpy array) where mP[i, j] is the previous segment for segment i at sample j.
        - mC: Minimum cost segmentation matrix (2D numpy array) where mS[i, j] is the minimum cost for segment i at sample j.
    Output:
        - lPath: List of indices representing the path from the first segment to the last segment.
    Example:
        ```python
        mP = np.array([[0, 1, 2], [1, 0, 1], [2, 1, 0]])
        mS = np.array([[0, 1, 2], [1, 0, 1], [2, 1, 0]])
        lPath = ExtractPath(mP, mS)
        print(lPath)
        ```
    Remarks:
        - This function assumes that the last segment is always valid and starts from there.
    """

    numRows = mK.shape[0]
    numCols = mK.shape[1]

    mS = np.zeros((numRows, 2), dtype = int) #<! (i, j)

    jj = np.argmin(mC[-1, :]) #<! Find the starting index of the last segment
    mS[-1, 0] = numRows - 1 #<! The last segment is the last row
    mS[-1, 1] = jj #<! The last segment is the last column

    for ii in range(numRows - 2, -1, -1):
        mS[ii, 0] = ii
        mS[ii, 1] = mS[ii + 1, 1] + mK[ii + 1, mS[ii + 1, 1]]
    
    return mS #<! Reverse the path to get it in correct order

In [ ]:
# Calculate the Cost Matrix
mC, mK = CalcCostMatrix(mE)

In [ ]:
# Extract the Path from `mP` Matrix
mS = ExtractSeamPath(mK, mC)

In [ ]:
# Plot the Cost Matrix and the Path

seamEnergy = np.sum(mE[mS[:, 0].astype(int), mS[:, 1].astype(int)]) #<! mC[mS[-1, 0], mS[-1, 1]] 

hF, hA = plt.subplots(figsize = (10, 6))
hA.imshow(mE)
hA.plot(mS[:, 1], mS[:, 0], color = 'magenta', linewidth = 1.5, alpha = 0.45, label = f'Optimal Seam: {seamEnergy:.2f}')
hA.legend()
hA.set_title('Energy Image and Optimal Seam Carve');


* <font color='brown'>(**#**)</font> For the data above, can the segments be inferred from the Cost Matrix?

* <font color='green'>(**@**)</font> Implement a faster solution by avoiding recalculation of the whole Energy Matrix $\boldsymbol{E}$ after each seam removal.

In [ ]:
def PlotDirectionMatrix( mK: np.ndarray, hA: plt.Axes ) -> plt.Axes:
    """
    Plots an array of directions {-1, 0, 1} as arrows on a 2D grid using matplotlib.
    Input:
        - mK: A 2D array of shape (m, n) with values in {-1, 0, 1}, where:
            - -1 → Top Left.
            -  0 → Top.
            - +1 → Top Right.
        - hA: The matplotlib axes to plot on.
    Output:
        - hA: The matplotlib axes with the plotted arrows.
    """

    numRows, numCols = mK.shape

    mY, mX = np.meshgrid(np.arange(numRows) + 0.5, np.arange(numCols) + 0.5, indexing = 'ij')

    # Define the direction vectors
    mU = mK                    #<! x component: -1, 0, or +1
    mV = np.full_like(mK, 1)  #<! y component is always -1 (pointing downward)

    mN = np.sqrt(np.square(mU) + np.square(mV)) / 0.5 #<! Normalize the vectors
    mU = mU / mN
    mV = mV / mN

    # Plot
    hA.quiver(mX, mY, mU, mV, angles = 'uv', scale_units = 'xy', scale=1, color = 'black')
    for ii in range(numRows):
        # Create vertical lines
        hA.axvline(x = ii, color = 'gray', linestyle = '--', alpha = 0.5)
    for jj in range(numCols):
        # Create horizontal lines
        hA.axhline(y = jj, color = 'gray', linestyle = '--', alpha = 0.5)
    hA.set_xticks(np.arange(numCols) + 0.5, labels = np.arange(numCols))
    hA.set_yticks(np.arange(numRows) + 0.5, labels = np.arange(numRows))
    hA.minorticks_off()
    hA.grid(False, 'major')
    hA.grid(False, 'minor')
    # hA.axis('equal')
    hA.set_xlim(0, numCols)
    hA.set_ylim(0, numRows)
    hA.invert_yaxis()

    return hA


In [ ]:
mK[:5, :5]

In [ ]:
hF, hA = plt.subplots(figsize = (4, 4))

hA = PlotDirectionMatrix(mK[:5, :5], hA)